In [4]:
import uuid
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import pandas as pd

In [5]:
def read_eval_results(results):
    with open(results, 'r') as json_file:
        json_list_str = list(json_file)
    json_data = dict()
    json_list = list()
    for json_str in json_list_str:
        result = json.loads(json_str)
        json_data[str(uuid.uuid4())] = result
        json_list.append(result)
    return json_list

In [129]:
FILE_PATH = "/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/1_0/Text generations/vllm_llama2_13b_1.0_repeat_copy_text.jsonl"
file_name = FILE_PATH.split('/')[-1]
BASE_PATH = '/'.join(FILE_PATH.split('/')[:-1])
GENERATION_PATH = BASE_PATH + "/" + file_name[:-6] + '_generation.pkl'
OUTPUT_PATH = BASE_PATH + "/" + "RESULT_JSONLS_TEXT_1.0" + "/" + file_name[:-6] + '_result.jsonl'

In [91]:
# INPUT_PATH = '/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/0_1/Text generations/vllm_llama2_13b_0.1_repeat_copy_text.jsonl'
# OUTPUT_PATH = "/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments/0_9_new/Text Generations/RESULT_JSONLS_TEXT_0.9/vllm_llama2_70b_0.9_repeat_copy_text.jsonl"

#### Load Data

In [111]:

data = read_eval_results(FILE_PATH)
len(data)

32

In [112]:
df = pd.DataFrame(data)

In [113]:
df.head()

,input,target,comment,generation
0,Repeat 5 times hello world,hello world hello world hello world hello worl...,LSTMs could solve this in 2014. Warmup example.,[[On repeating the words hello world five time...
1,repeat the word cat four times. After the seco...,cat cat meow cat cat,Let's make it more challenging with basic inde...,[[On repeating the word cat four times we get:...
2,"Repeat the word dog four times, but halfway th...",dog dog woof dog dog,Now let's add some basic inference on top of i...,[[On repeating the word dog four times we get:...
3,"Repeat all the world seven times, and after ev...",all the world all the world is a stage all the...,NaN,[[On repeating the word all the world seven ti...
4,"Say hungry three times, then hippo two times, ...",hungry hungry hungry hippo hippo feed me feed ...,NaN,[[On repeating the word hungry three times we ...


In [114]:
all_generations = []
for generations in df['generation']:
    generations = generations[0]
    processed_generations = []
    for generation in generations:
        processed_generations.append(generation.split('\n\n')[0])
    all_generations.append(processed_generations)

In [115]:
len(all_generations)

32

In [116]:
for generations in all_generations:
    for generation in generations:
        assert generation == generation.split('\n\n')[0]

In [117]:
import pickle
with open(GENERATION_PATH, "wb") as f:
    pickle.dump(all_generations, f)

#### Answer Extraction Techniques 

In [118]:

import re

def extract_number(string):
    pattern = r'The answer is:? (\d+)'
    match = re.search(pattern, string)
    if match:
        return int(match.group(1))
    else:
        return None
    
def extract_text(string):
    pattern = r'The answer is:(.*)'
    match = re.search(pattern, string)
    
    if match:
        result = match.group(1).strip()
        return result
    else:
        return None

def extract_date(string):
    pattern = r"answer is:? (\d{2}/\d{2}/\d{4})"
    match = re.search(pattern, string)
    if match:
        return match.group(1)
    else:
        return None


#### Repeat Copy [String Centric] extraction

In [119]:
all_candidates_ans = []
for i,r in df.iterrows():
    ans =[]
    for j in r['generation'][0]:
        val = j.split('\n\n')[0]
        extract = extract_text(val)
        if extract == None:
            ans.append("non_extractable")
        else:
            ans.append(extract)
    all_candidates_ans.append(ans)     
    


In [120]:
all_candidates_ans[:1]

[['hello world hello world hello world hello world hello world',
  'hello world hello world hello world hello world hello world',
  '',
  'hello world hello world hello world hello world hello world',
  'hello world hello world hello world hello world hello world',
  'hello world hello world hello world hello world hello world hello world',
  'hello world hello world hello world hello world hello world',
  'hello2 hello2 world3 hello2 world3 hello2 world3 hello2',
  'non_extractable',
  'hello hello world hello hello world hello hello world hello hello world']]

#### Append the highest frequency Answer

In [121]:
from collections import Counter
max_a=[] #Maximum occuring value in list ---- answer
cal_all=[] # Calibration dictionary

for l in all_candidates_ans :
    unique, counts = np.unique(np.array(l), return_counts=True)
    d = dict(np.asarray((unique, counts / len(l))).T)
    cal_all.append(d)
    key_counts = Counter(l)

    # Find the key with the highest count
    try:
        most_common_key = key_counts.most_common(1)[0][0]
    except:
        most_common_key = None
    max_a.append(most_common_key)

#### Accuracy 

In [122]:

def accuracy(list1, list2):
    matches =0
    for i in range(len(list1)):
        if list1[i] == list2[i]:
            matches += 1

# Calculate the accuracy as a percentage
    accuracy = (matches / len(list1)) * 100

    return accuracy

In [123]:
df.head()

,input,target,comment,generation
0,Repeat 5 times hello world,hello world hello world hello world hello worl...,LSTMs could solve this in 2014. Warmup example.,[[On repeating the words hello world five time...
1,repeat the word cat four times. After the seco...,cat cat meow cat cat,Let's make it more challenging with basic inde...,[[On repeating the word cat four times we get:...
2,"Repeat the word dog four times, but halfway th...",dog dog woof dog dog,Now let's add some basic inference on top of i...,[[On repeating the word dog four times we get:...
3,"Repeat all the world seven times, and after ev...",all the world all the world is a stage all the...,NaN,[[On repeating the word all the world seven ti...
4,"Say hungry three times, then hippo two times, ...",hungry hungry hungry hippo hippo feed me feed ...,NaN,[[On repeating the word hungry three times we ...


In [124]:
target = df['target'].tolist()


In [125]:
print(accuracy(target, max_a))

31.25


#### To calculate calibration 

In [126]:
result_jsonl = []
for calibration_result,answer, target in zip(cal_all, max_a, target):
    json_result = dict()
    json_result['calibration_result'] = [calibration_result]
    json_result['answer'] = answer
    json_result['target'] = target
    json_result['score'] = 1 if answer == target else 0
    result_jsonl.append(json_result)

In [127]:
result_jsonl[:1]

[{'calibration_result': [{'': '0.1',
    'hello hello world hello hello world hello hello world hello hello world': '0.1',
    'hello world hello world hello world hello world hello world': '0.5',
    'hello world hello world hello world hello world hello world hello world': '0.1',
    'hello2 hello2 world3 hello2 world3 hello2 world3 hello2': '0.1',
    'non_extractable': '0.1'}],
  'answer': 'hello world hello world hello world hello world hello world',
  'target': 'hello world hello world hello world hello world hello world',
  'score': 1}]

In [130]:
import json
with open(OUTPUT_PATH, "w") as f:
    json_obj = json.dumps(result_jsonl)
    f.write(json_obj)